This notebook demonstrates advanced SQL techniques like CASE statements, subqueries, WITH clauses, and window functions through analysis of the European Soccer Database. The soccer database contains team, match, and player data for top European leagues from 2008-2016.

After importing necessary libraries like pandas and sqlite3, SQL queries are constructed to explore the database. Techniques like conditional logic in CASE statements, nesting subqueries, common table expressions with WITH clause, and analytic functions like ranking are applied.

The goals are to 
- gain insights into the soccer data, and 
- exemplify effective use of advanced SQL constructs for performing complex analysis. The notebook serves both as a practical guide for learning SQL and a showcase of impactful techniques using a real-world sports dataset.

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

conn = sqlite3.connect('/kaggle/input/soccer/database.sqlite')

In [2]:
tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables


,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [3]:
match_table = pd.read_sql("""SELECT *
                        FROM Match;""", conn)
match_table


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
Player_table = pd.read_sql("""SELECT *
                        FROM Player;""", conn)
Player_table

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
...,...,...,...,...,...,...,...
11055,11071,26357,Zoumana Camara,2488,1979-04-03 00:00:00,182.88,168
11056,11072,111182,Zsolt Laczko,164680,1986-12-18 00:00:00,182.88,176
11057,11073,36491,Zsolt Low,111191,1979-04-29 00:00:00,180.34,154
11058,11074,35506,Zurab Khizanishvili,47058,1981-10-06 00:00:00,185.42,172


In [5]:
League_table = pd.read_sql("""SELECT *
                        FROM League;""", conn)
League_table

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


In [6]:
Country_table = pd.read_sql("""SELECT *
                        FROM Country;""", conn)
Country_table

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [7]:
Team_table = pd.read_sql("""SELECT *
                        FROM Country;""", conn)
Team_table

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [8]:
playerattributes_table = pd.read_sql("""SELECT *
                        FROM Player_Attributes;""", conn)
playerattributes_table

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,183974,102359,39902,2009-08-30 00:00:00,83.0,85.0,right,medium,low,84.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,183975,102359,39902,2009-02-22 00:00:00,78.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,183976,102359,39902,2008-08-30 00:00:00,77.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30 00:00:00,78.0,81.0,right,medium,low,74.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [9]:
teamattributes_table = pd.read_sql("""SELECT *
                        FROM Team_Attributes;""", conn)
teamattributes_table

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,1454,15005,10000,2011-02-22 00:00:00,52,Balanced,NaN,Little,52,Mixed,...,53,Normal,Organised,46,Medium,48,Press,53,Normal,Cover
1454,1455,15005,10000,2012-02-22 00:00:00,54,Balanced,NaN,Little,51,Mixed,...,50,Normal,Organised,44,Medium,55,Press,53,Normal,Cover
1455,1456,15005,10000,2013-09-20 00:00:00,54,Balanced,NaN,Little,51,Mixed,...,32,Little,Organised,44,Medium,58,Press,37,Normal,Cover
1456,1457,15005,10000,2014-09-19 00:00:00,54,Balanced,42.0,Normal,51,Mixed,...,32,Little,Organised,44,Medium,58,Press,37,Normal,Cover


Lets think as a football fan!

I want to take a look at which matches were played in a particular league and during a particular season and I probably want to take a look at scores as well.

To get the the desired results, we have to join match table with country table and then with league tabel then to the team table

Some of the fields in the different tables have same names. Renaming using AS in the query


In [10]:
detailed_matches = pd.read_sql("""
SELECT Match.id, 
Country.name AS country_name, 
League.name AS league_name, 
season, 
date,
HT.team_long_name AS  home_team,
AT.team_long_name AS away_team,
home_team_goal, 
away_team_goal                                        
FROM Match
LEFT JOIN Country on Country.id = Match.country_id
LEFT JOIN League on League.id = Match.league_id
LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
WHERE country_name = 'England'
AND season = '2015/2016'
ORDER by date
LIMIT 10;""", conn)
detailed_matches

,id,country_name,league_name,season,date,home_team,away_team,home_team_goal,away_team_goal
0,4390,England,England Premier League,2015/2016,2015-08-08 00:00:00,Bournemouth,Aston Villa,0,1
1,4391,England,England Premier League,2015/2016,2015-08-08 00:00:00,Chelsea,Swansea City,2,2
2,4392,England,England Premier League,2015/2016,2015-08-08 00:00:00,Everton,Watford,2,2
3,4393,England,England Premier League,2015/2016,2015-08-08 00:00:00,Leicester City,Sunderland,4,2
4,4394,England,England Premier League,2015/2016,2015-08-08 00:00:00,Manchester United,Tottenham Hotspur,1,0
5,4396,England,England Premier League,2015/2016,2015-08-08 00:00:00,Norwich City,Crystal Palace,1,3
6,4389,England,England Premier League,2015/2016,2015-08-09 00:00:00,Arsenal,West Ham United,0,2
7,4395,England,England Premier League,2015/2016,2015-08-09 00:00:00,Newcastle United,Southampton,2,2
8,4397,England,England Premier League,2015/2016,2015-08-09 00:00:00,Stoke City,Liverpool,0,1
9,4398,England,England Premier League,2015/2016,2015-08-10 00:00:00,West Bromwich Albion,Manchester City,0,3


As a fan, i want to take a look at match results of the ream I follow.

In [11]:
tot = pd.read_sql("""
SELECT m.date,
HT.team_long_name AS  home_team,
AT.team_long_name AS away_team,
home_team_goal, 
away_team_goal, 
CASE WHEN m.home_team_api_id = 8586 and m.home_team_goal > m.away_team_goal THEN 'Spurs won'
WHEN m.home_team_api_id = 8586 and  m.home_team_goal < m.away_team_goal THEN 'Spurs lost'
WHEN m.away_team_api_id = 8586 and m.home_team_goal < m.away_team_goal THEN 'Spurs won'
WHEN m.away_team_api_id = 8586 and m.home_team_goal > m.away_team_goal THEN 'Spurs lost'
ELSE 'Tie' END AS outcome
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
Order by 1;""", conn)
tot

,date,home_team,away_team,home_team_goal,away_team_goal,outcome
0,2008-08-16 00:00:00,Middlesbrough,Tottenham Hotspur,2,1,Spurs lost
1,2008-08-23 00:00:00,Tottenham Hotspur,Sunderland,1,2,Spurs lost
2,2008-08-31 00:00:00,Chelsea,Tottenham Hotspur,1,1,Tie
3,2008-09-15 00:00:00,Tottenham Hotspur,Aston Villa,1,2,Spurs lost
4,2008-09-21 00:00:00,Tottenham Hotspur,Wigan Athletic,0,0,Tie
...,...,...,...,...,...,...
299,2016-04-18 00:00:00,Stoke City,Tottenham Hotspur,0,4,Spurs won
300,2016-04-25 00:00:00,Tottenham Hotspur,West Bromwich Albion,1,1,Tie
301,2016-05-02 00:00:00,Chelsea,Tottenham Hotspur,2,2,Tie
302,2016-05-08 00:00:00,Tottenham Hotspur,Southampton,1,2,Spurs lost


Now lets create a view for how my fav team performed each season.

In [12]:
totout = pd.read_sql("""
SELECT
m.season,
CASE WHEN m.home_team_api_id = 8586 and m.home_team_goal > m.away_team_goal THEN 'Spurs won'
WHEN m.home_team_api_id = 8586 and  m.home_team_goal < m.away_team_goal THEN 'Spurs lost'
WHEN m.away_team_api_id = 8586 and m.home_team_goal < m.away_team_goal THEN 'Spurs won'
WHEN m.away_team_api_id = 8586 and m.home_team_goal > m.away_team_goal THEN 'Spurs lost'
ELSE 'Tie' END AS outcome,
count (m.date) AS count
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
GROUP BY m.season,outcome;""", conn)
totout

,season,outcome,count
0,2008/2009,Spurs lost,15
1,2008/2009,Spurs won,14
2,2008/2009,Tie,9
3,2009/2010,Spurs lost,10
4,2009/2010,Spurs won,21
5,2009/2010,Tie,7
6,2010/2011,Spurs lost,8
7,2010/2011,Spurs won,16
8,2010/2011,Tie,14
9,2011/2012,Spurs lost,9


Lets create a view for how many goals a team scored across a season.

In [13]:
GoalsBySeason = pd.read_sql("""
SELECT
m.season,
SUM(CASE WHEN m.home_team_api_id = 8586  THEN home_team_goal END) AS home_goals,
SUM(CASE WHEN m.away_team_api_id = 8586  THEN away_team_goal END) AS away_goals
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
GROUP BY m.season;""", conn)
GoalsBySeason

,season,home_goals,away_goals
0,2008/2009,21,24
1,2009/2010,40,27
2,2010/2011,30,25
3,2011/2012,39,27
4,2012/2013,29,37
5,2013/2014,30,25
6,2014/2015,31,27
7,2015/2016,35,34


Lets make a view for average goals by season

In [14]:
AvgGoalsBySeason = pd.read_sql("""
SELECT
m.season,
ROUND(AVG(CASE WHEN m.home_team_api_id = 8586  THEN home_team_goal END),2) AS average_home_goals,
ROUND(AVG(CASE WHEN m.home_team_api_id = 8586  THEN away_team_goal END),2) AS average_away_goals
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
GROUP BY m.season;""", conn)
AvgGoalsBySeason

,season,average_home_goals,average_away_goals
0,2008/2009,1.11,0.53
1,2009/2010,2.11,0.63
2,2010/2011,1.58,1.00
3,2011/2012,2.05,0.89
4,2012/2013,1.53,0.95
5,2013/2014,1.58,1.21
6,2014/2015,1.63,1.26
7,2015/2016,1.84,0.79


In [15]:
AvgGoalsBySeason = pd.read_sql("""
SELECT
m.season,
ROUND(AVG(CASE WHEN m.home_team_api_id = 8586  THEN home_team_goal END),2) AS average_home_goals,
ROUND(AVG(CASE WHEN m.home_team_api_id = 8586  THEN away_team_goal END),2) AS average_away_goals
FROM match m
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
GROUP BY m.season;""", conn)
AvgGoalsBySeason

,season,average_home_goals,average_away_goals
0,2008/2009,1.11,0.53
1,2009/2010,2.11,0.63
2,2010/2011,1.58,1.00
3,2011/2012,2.05,0.89
4,2012/2013,1.53,0.95
5,2013/2014,1.58,1.21
6,2014/2015,1.63,1.26
7,2015/2016,1.84,0.79


Lets calculate a view for what percentage of home and away matches fav team won

In [16]:
AvgGoalsBySeasonPCT = pd.read_sql(
"""
SELECT
m.season,
ROUND(AVG(CASE WHEN m.home_team_api_id = 8586 AND home_team_goal > away_team_goal THEN 1
WHEN m.home_team_api_id = 8586 AND home_team_goal < away_team_goal THEN 0 END),2) AS  pct_homewins,
ROUND(AVG(CASE WHEN m.away_team_api_id = 8586 AND away_team_goal > home_team_goal THEN 1
WHEN m.home_team_api_id = 8586 AND away_team_goal < home_team_goal THEN 0 END),2) AS  pct_awaywins
FROM match m 
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.home_team_api_id = 8586 or m.away_team_api_id = 8586
GROUP BY m.season;""", conn)
AvgGoalsBySeasonPCT

,season,pct_homewins,pct_awaywins
0,2008/2009,0.71,0.29
1,2009/2010,0.82,0.33
2,2010/2011,0.90,0.44
3,2011/2012,0.81,0.35
4,2012/2013,0.79,0.48
5,2013/2014,0.69,0.48
6,2014/2015,0.63,0.47
7,2015/2016,0.77,0.47


Lets create a view for what percentage of matches end in draw across different countries and leagues

To keep the comparison focussed lets just compare seasons 2013/14 and 2014/15.



In [17]:
perdraw = pd.read_sql("""
SELECT c.name AS country,
ROUND(AVG(CASE WHEN m.season='2013/2014' AND m.home_team_goal = m.away_team_goal THEN 1
 WHEN m.season='2013/2014' AND m.home_team_goal != m.away_team_goal THEN 0
END),2) AS pct_ties_2013_2014,
ROUND(AVG(CASE WHEN m.season='2014/2015' AND m.home_team_goal = m.away_team_goal THEN 1
 WHEN m.season='2014/2015' AND m.home_team_goal != m.away_team_goal THEN 0
END),2) AS pct_ties_2014_2015
FROM country AS c
LEFT JOIN match AS m
ON c.id = m.country_id
GROUP BY country
;""", conn)
perdraw

,country,pct_ties_2013_2014,pct_ties_2014_2015
0,Belgium,0.17,0.25
1,England,0.21,0.24
2,France,0.28,0.23
3,Germany,0.21,0.27
4,Italy,0.24,0.32
5,Netherlands,0.27,0.24
6,Poland,0.30,0.28
7,Portugal,0.25,0.28
8,Scotland,0.22,0.19
9,Spain,0.23,0.24


In the 2014/2015 season, Italy had the highest proportion of draws (almost 1/3) and Scotland had the lowest (about 1/5)

Following on from this, what league had the most number of goals, averaging out across all the matches within that league?

In [18]:
leagues_by_avg_goal = pd.read_sql("""
SELECT 
l.name AS league,
ROUND(AVG(m.home_team_goal + m.away_team_goal),2) AS avg_goals
FROM league as L
LEFT JOIN match AS m
ON l.id = m.country_id
WHERE m.season = '2014/2015'
GROUP BY league
;""", conn)
leagues_by_avg_goal

,league,avg_goals
0,Belgium Jupiler League,2.78
1,England Premier League,2.57
2,France Ligue 1,2.49
3,Germany 1. Bundesliga,2.75
4,Italy Serie A,2.69
5,Netherlands Eredivisie,3.08
6,Poland Ekstraklasa,2.62
7,Portugal Liga ZON Sagres,2.49
8,Scotland Premier League,2.57
9,Spain LIGA BBVA,2.66


We can see that the Netherlands had the highest average number of goals with 3.08.

 what was the average number of goals,across all leagues, during the 2014/15 season.

In [19]:
avg_goal1415 = pd.read_sql("""
SELECT 
ROUND(AVG(m.home_team_goal + m.away_team_goal),2) AS avg_goals
FROM match AS m
WHERE m.season = '2014/2015';""", conn)
avg_goal1415

,avg_goals
0,2.68


Number of matches in the French league, where more goals were scored than this average.

In [20]:
Higher_than_avg_goals1415 = pd.read_sql("""
SELECT 
HT.team_long_name AS  home_team,
AT.team_long_name AS away_team,
m.home_team_goal,
m.away_team_goal
FROM match AS m
LEFT JOIN League as l
ON l.id = m.country_id
LEFT JOIN Team AS HT on HT.team_api_id = m.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = m.away_team_api_id
WHERE m.season = '2014/2015'
AND l.name = 'France Ligue 1'
AND (m.home_team_goal + m.away_team_goal) >
(SELECT 
    AVG(m.home_team_goal + m.away_team_goal) AS avg_goals
    FROM match AS m
WHERE m.season = '2014/2015');""", conn)
Higher_than_avg_goals1415

,home_team,away_team,home_team_goal,away_team_goal
0,Stade de Reims,Paris Saint-Germain,2,2
1,SC Bastia,Olympique de Marseille,3,3
2,Évian Thonon Gaillard FC,SM Caen,0,3
3,AS Monaco,FC Lorient,1,2
4,OGC Nice,Toulouse FC,3,2
...,...,...,...,...
165,FC Metz,Stade de Reims,3,0
166,Montpellier Hérault SC,En Avant de Guingamp,2,1
167,SM Caen,Olympique de Marseille,1,2
168,Évian Thonon Gaillard FC,FC Metz,3,0


lets pull all the matches in the database, but then only those matches where more than 10 goals were scored.

In [21]:
matchesabove10 = pd.read_sql("""
SELECT 
country,date,total_goals
FROM 
(SELECT 
c.name as country,
m.date,
m.home_team_goal,
m.away_team_goal,
(m.home_team_goal + m.away_team_goal) as total_goals
from match AS m
left join country as c
ON m.country_id = c.id) AS subquery
WHERE 
total_goals >=10
    ;""", conn)
matchesabove10

,country,date,total_goals
0,England,2009-11-22 00:00:00,10
1,England,2011-08-28 00:00:00,10
2,England,2012-12-29 00:00:00,10
3,England,2013-05-19 00:00:00,10
4,France,2009-11-08 00:00:00,10
5,Germany,2013-03-30 00:00:00,11
6,Netherlands,2010-10-24 00:00:00,10
7,Netherlands,2011-11-06 00:00:00,10
8,Scotland,2010-05-05 00:00:00,12
9,Spain,2013-10-30 00:00:00,10


by using a subquery within the SELECT clause, the total goals scored in each 2014/15 match can be benchmarked against the average goals per match across all seasons. This demonstrates using subqueries to derive comparison metrics within a result set.

In [22]:
TotalGoalsVAvG = pd.read_sql("""
SELECT 
date,
(m.home_team_goal + m.away_team_goal) AS total_goals,
(SELECT ROUND(AVG(m.home_team_goal + m.away_team_goal),2)
FROM match AS m
WHERE season = '2014/2015') AS overall_avg
FROM match m
WHERE m.season = '2014/2015'
    ;""", conn)
TotalGoalsVAvG

,date,total_goals,overall_avg
0,2014-07-25 00:00:00,3,2.68
1,2014-07-26 00:00:00,2,2.68
2,2014-07-26 00:00:00,0,2.68
3,2014-07-26 00:00:00,2,2.68
4,2014-07-26 00:00:00,1,2.68
...,...,...,...
3320,2014-09-23 00:00:00,4,2.68
3321,2014-09-23 00:00:00,5,2.68
3322,2014-09-24 00:00:00,3,2.68
3323,2014-09-24 00:00:00,0,2.68


Lets calculate the average total of goals scored within each league

Lets calculate avg total of goals scored in each league and also use a sub query in the select statement to subtract the overall average from each leagues' average. Through this we get a sense of where each league stands in relation to the average.

In [23]:
goalsdiff = pd.read_sql("""
Select 
l.name as league,
round((avg(m.home_team_goal + m.away_team_goal)),2) as avg_goals,
round(avg(m.home_team_goal + m.away_team_goal)
-((Select avg(home_team_goal + away_team_goal) from match WHERE season = '2014/2015')),2) as diff 
FROM league as l
LEFT JOIN match as m
ON l.country_id = m.country_id
WHERE m.season = '2014/2015'
Group by l.name
    ;""", conn)
goalsdiff

,league,avg_goals,diff
0,Belgium Jupiler League,2.78,0.11
1,England Premier League,2.57,-0.11
2,France Ligue 1,2.49,-0.18
3,Germany 1. Bundesliga,2.75,0.08
4,Italy Serie A,2.69,0.01
5,Netherlands Eredivisie,3.08,0.40
6,Poland Ekstraklasa,2.62,-0.06
7,Portugal Liga ZON Sagres,2.49,-0.18
8,Scotland Premier League,2.57,-0.10
9,Spain LIGA BBVA,2.66,-0.02


lets create a CTE which pulls all the matches where 10 or more goals were scored. Then lets join that information onto the league table, so we get a count of how matches have occurred in each league

In [24]:
HighScoreMatchesByLeague = pd.read_sql("""
With big_game AS (
Select 
league_id
,id
from match 
WHERE home_team_goal + away_team_goal >= 10)
Select
l.name as league,
COUNT(big_game.id) as High_Score_Matches
FROM league as l
LEFT JOIN big_game
ON l.id = big_game.league_id
GROUP BY league
    ;""", conn)
HighScoreMatchesByLeague

,league,High_Score_Matches
0,Belgium Jupiler League,0
1,England Premier League,4
2,France Ligue 1,1
3,Germany 1. Bundesliga,1
4,Italy Serie A,0
5,Netherlands Eredivisie,2
6,Poland Ekstraklasa,0
7,Portugal Liga ZON Sagres,0
8,Scotland Premier League,1
9,Spain LIGA BBVA,5


There are leagues who never had a single match where 10 or more goals were scored. At the other extreme, spains LIGA BBVA had the highest number of matches where this happened.

Lets create a query that sums up the total number of goals that were scored in matches and then joining that information to an outer query that sums up the information by a dimension only available outside of CTE, lets say league name.

In [25]:
AvgGoalsByleague = pd.read_sql("""
With match_list as
(
Select 
country_id
,(home_team_goal + away_team_goal) AS goals
from match
)
Select name
,ROUND(AVG (goals),2) as Average_Goals
FROM league
LEFT JOIN match_list 
ON league.id = match_list.country_id
Group by name

 ;""", conn)

AvgGoalsByleague

,name,Average_Goals
0,Belgium Jupiler League,2.80
1,England Premier League,2.71
2,France Ligue 1,2.44
3,Germany 1. Bundesliga,2.90
4,Italy Serie A,2.62
5,Netherlands Eredivisie,3.08
6,Poland Ekstraklasa,2.43
7,Portugal Liga ZON Sagres,2.53
8,Scotland Premier League,2.63
9,Spain LIGA BBVA,2.77


Lets create a query that shows the total goals scored in every single match and also calculate the average number of goals over the season

In [26]:
Goals = pd.read_sql("""
SELECT 
date
,(m.home_team_goal +m.away_team_goal) AS total_goals,
ROUND(AVG(home_team_goal + away_team_goal) OVER (),2) AS overall_avg
FROM Match m
WHERE season = '2014/2015';""", conn)
Goals

,date,total_goals,overall_avg
0,2014-07-25 00:00:00,3,2.68
1,2014-07-26 00:00:00,2,2.68
2,2014-07-26 00:00:00,0,2.68
3,2014-07-26 00:00:00,2,2.68
4,2014-07-26 00:00:00,1,2.68
...,...,...,...
3320,2014-09-23 00:00:00,4,2.68
3321,2014-09-23 00:00:00,5,2.68
3322,2014-09-24 00:00:00,3,2.68
3323,2014-09-24 00:00:00,0,2.68



Lets use an OVER clause, combined with a RANK function, which will rank the results over our result set.

In [27]:
leagues_by_avgG = pd.read_sql("""
SELECT 
League.name AS league_name,
ROUND(AVG(home_team_goal + away_team_goal),2) as avg_goals
FROM Match
LEFT JOIN League on League.id = Match.league_id
GROUP BY league_name                      
;""", conn)
leagues_by_avgG

,league_name,avg_goals
0,Belgium Jupiler League,2.80
1,England Premier League,2.71
2,France Ligue 1,2.44
3,Germany 1. Bundesliga,2.90
4,Italy Serie A,2.62
5,Netherlands Eredivisie,3.08
6,Poland Ekstraklasa,2.43
7,Portugal Liga ZON Sagres,2.53
8,Scotland Premier League,2.63
9,Spain LIGA BBVA,2.77


Netherlands Eredivisie had the highest number of average total goals out of all the leagues. To see how the leagues rank against each other on this, we could easily order the query results by this figure. However maybe we need to keep the leagues in their alphabetical order. To still see this rank, we can use a RANK function, and combine it with an OVER clause

In [28]:
leagues_by_avgGRank = pd.read_sql("""
SELECT 
League.name AS league_name,
ROUND(AVG(home_team_goal + away_team_goal),2) as avg_goals,
RANK () OVER (ORDER BY  AVG (home_team_goal + away_team_goal)DESC) AS league_rank
FROM Match
LEFT JOIN League on League.id = Match.league_id
GROUP BY league_name
ORDER BY League_name;""", conn)
leagues_by_avgGRank

,league_name,avg_goals,league_rank
0,Belgium Jupiler League,2.80,4
1,England Premier League,2.71,6
2,France Ligue 1,2.44,10
3,Germany 1. Bundesliga,2.90,3
4,Italy Serie A,2.62,8
5,Netherlands Eredivisie,3.08,1
6,Poland Ekstraklasa,2.43,11
7,Portugal Liga ZON Sagres,2.53,9
8,Scotland Premier League,2.63,7
9,Spain LIGA BBVA,2.77,5


lets write a query that once again, finds the average total goals but this time, that average will be partitioned by a category, in this case by season. This allows us to see that average right next to the data the generates it.

In [29]:
leagues_by_season = pd.read_sql("""SELECT
Match.date,                                       
League.name AS league_name,
season,
home_team_goal + away_team_goal as Total_Goals,
ROUND(AVG ( home_team_goal + away_team_goal) OVER (partition by season),2) as season_avg
FROM Match
LEFT JOIN League 
ON League.id = Match.league_id;""", conn)
leagues_by_season

,date,league_name,season,Total_Goals,season_avg
0,2008-08-17 00:00:00,Belgium Jupiler League,2008/2009,2,2.61
1,2008-08-16 00:00:00,Belgium Jupiler League,2008/2009,0,2.61
2,2008-08-16 00:00:00,Belgium Jupiler League,2008/2009,3,2.61
3,2008-08-17 00:00:00,Belgium Jupiler League,2008/2009,5,2.61
4,2008-08-16 00:00:00,Belgium Jupiler League,2008/2009,4,2.61
...,...,...,...,...,...
25974,2015-09-22 00:00:00,Switzerland Super League,2015/2016,1,2.75
25975,2015-09-23 00:00:00,Switzerland Super League,2015/2016,3,2.75
25976,2015-09-23 00:00:00,Switzerland Super League,2015/2016,2,2.75
25977,2015-09-22 00:00:00,Switzerland Super League,2015/2016,0,2.75


lets add league to the existing partition and so now we will be calculating the average number of goals within each season, within each league.

In [30]:
leagues_by_season_leag = pd.read_sql("""SELECT
Match.date,                                       
League.name AS league_name,
season,
home_team_goal + away_team_goal as Total_Goals,
AVG ( home_team_goal + away_team_goal) OVER (partition by season,League.name) as season_league_avg
FROM Match
LEFT JOIN League 
ON League.id = Match.league_id;""", conn)
leagues_by_season_leag

,date,league_name,season,Total_Goals,season_league_avg
0,2008-08-17 00:00:00,Belgium Jupiler League,2008/2009,2,2.794118
1,2008-08-16 00:00:00,Belgium Jupiler League,2008/2009,0,2.794118
2,2008-08-16 00:00:00,Belgium Jupiler League,2008/2009,3,2.794118
3,2008-08-17 00:00:00,Belgium Jupiler League,2008/2009,5,2.794118
4,2008-08-16 00:00:00,Belgium Jupiler League,2008/2009,4,2.794118
...,...,...,...,...,...
25974,2015-09-22 00:00:00,Switzerland Super League,2015/2016,1,3.144444
25975,2015-09-23 00:00:00,Switzerland Super League,2015/2016,3,3.144444
25976,2015-09-23 00:00:00,Switzerland Super League,2015/2016,2,3.144444
25977,2015-09-22 00:00:00,Switzerland Super League,2015/2016,0,3.144444


lets take a look at each match played bye-bye a(single) team has played every season and have the average number of home and away goals the team scored on average within that season and month.

In [31]:
leagues_by_season_month_spurs = pd.read_sql("""
SELECT
Match.date,
strftime('%m', Match.date) as Month,
season,
HT.team_long_name AS  home_team,
AT.team_long_name AS away_team,
home_team_goal, 
away_team_goal,
CASE WHEN  Match.home_team_api_id = 8586 THEN 'home'
ELSE 'away' END as Home_Or_Away,
AVG (home_team_goal) OVER (partition by season,strftime('%m', Match.date)) as season_month_home_avg,
AVG (away_team_goal) OVER (partition by season,strftime('%m', Match.date)) as season_month_away_avg
FROM Match
LEFT JOIN League 
ON League.id = Match.league_id
LEFT JOIN Team AS HT on HT.team_api_id = Match.home_team_api_id
LEFT JOIN Team AS AT on AT.team_api_id = Match.away_team_api_id
WHERE Match.home_team_api_id = 8586 or Match.away_team_api_id = 8586
;""", conn)
leagues_by_season_month_spurs

,date,Month,season,home_team,away_team,home_team_goal,away_team_goal,Home_Or_Away,season_month_home_avg,season_month_away_avg
0,2009-01-11 00:00:00,01,2008/2009,Wigan Athletic,Tottenham Hotspur,1,0,away,2.0,1.0
1,2009-01-18 00:00:00,01,2008/2009,Tottenham Hotspur,Portsmouth,1,1,home,2.0,1.0
2,2009-01-27 00:00:00,01,2008/2009,Tottenham Hotspur,Stoke City,3,1,home,2.0,1.0
3,2009-01-31 00:00:00,01,2008/2009,Bolton Wanderers,Tottenham Hotspur,3,2,away,2.0,1.0
4,2009-02-08 00:00:00,02,2008/2009,Tottenham Hotspur,Arsenal,0,0,home,0.5,1.0
...,...,...,...,...,...,...,...,...,...,...
299,2015-12-05 00:00:00,12,2015/2016,West Bromwich Albion,Tottenham Hotspur,1,1,away,1.2,1.4
300,2015-12-13 00:00:00,12,2015/2016,Tottenham Hotspur,Newcastle United,1,2,home,1.2,1.4
301,2015-12-19 00:00:00,12,2015/2016,Southampton,Tottenham Hotspur,0,2,away,1.2,1.4
302,2015-12-26 00:00:00,12,2015/2016,Tottenham Hotspur,Norwich City,3,0,home,1.2,1.4
